# Notebook purpose

- Discover and prioritize high‑volume instruments and curate per‑day intraday CSVs for `{TICKER}`.
- Fetch or validate RAW files into `HIGH VOLUME STOCKS/RAW_DATA` following the naming: `{TICKER}_hist_data_tr_YYYY-MM-DD.csv`.
- Provide quick data sanity checks (file counts, date span, sample rows) before merging.

Run when
- Starting a new instrument or refreshing historical raw intraday data.

Inputs
- Data source/API credentials (if applicable) and target data directory.

Outputs
- Per‑day CSVs under `HIGH VOLUME STOCKS/RAW_DATA/` with the standardized naming used by downstream notebooks.

Tip
- Set TICKER = 'RELIND' to target a specific instrument

In [ ]:
# Environment info (interpreter and key libs)
import sys
print('Interpreter:', sys.executable)

import pandas as pd
print('pandas version:', pd.__version__)

import numpy as np
print('numpy version:', np.__version__)

try:
    import sklearn
    print('sklearn version:', sklearn.__version__)
except Exception as e:
    print('sklearn not available:', e)

try:
    import lightgbm as lgb
    print('lightgbm version:', lgb.__version__)
except Exception as e:
    print('lightgbm not available:', e)

# Extra env info
try:
    import tensorflow as tf
    print('tensorflow version:', tf.__version__)
except Exception as e:
    print('tensorflow not available:', e)

try:
    import xgboost as xgb
    print('xgboost version:', xgb.__version__)
except Exception as e:
    print('xgboost not available:', e)

try:
    import statsmodels
    print('statsmodels version:', statsmodels.__version__)
except Exception as e:
    print('statsmodels not available:', e)

In [ ]:
TICKER = "ICISEN"

# Centralized path variables (moved to top)
# Use an f+raw string so {TICKER} is interpolated and backslashes are preserved on Windows
output_data_dump_folder = fr"C:\Users\Admin\My_Projects\finance\HIGH VOLUME STOCKS\{TICKER}"
session_timestamp_file = r"C:\Users\Admin\My_Projects\ICICI_Direct_Analysis\Session_Key\Timestamp.txt"
session_key_storage_dir = r"C:\Users\Admin\My_Projects\ICICI_Direct_Analysis\Session_Key"
webdriver_path = r"C:\Windows\System32\chromedriver.exe"
login_url = "https://api.icicidirect.com/apiuser/home"



In [ ]:
# Path sanity check (run after the top config cell)
import os, glob

# Ensure the output folder and RAW_DATA subfolder exist so downstream saves won't fail
os.makedirs(output_data_dump_folder, exist_ok=True)


print("[Sanity] high_volume_stocks")
print(f"output_data_dump_folder: {output_data_dump_folder} | dir: {os.path.isdir(output_data_dump_folder)}")
print(f"session_key_storage_dir: {session_key_storage_dir} | dir: {os.path.isdir(session_key_storage_dir)}")
print(f"session_timestamp_file: {session_timestamp_file} | file: {os.path.isfile(session_timestamp_file)}")
print(f"webdriver_path: {webdriver_path} | file: {os.path.isfile(webdriver_path)}")



In [15]:

# ---- Start of imports ----
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import openpyxl
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from datetime import datetime, date, timedelta, time, timezone
from breeze_connect import BreezeConnect
import pytz

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
import time as tm

import smtplib

# MIMEMultipart send emails with both text content and attachments.
from email.mime.multipart import MIMEMultipart
# MIMEText for creating body of the email message.
from email.mime.text import MIMEText
# MIMEApplication attaching application-specific data (like CSV files) to email messages.
from email.mime.application import MIMEApplication
#---- End of imports ----

# --- Global Variables ---
global today_date_str
global yesterday_date_str
global current_price
global df
global s_token
global inv_df
global pos_df
global ema_rsi_df

# Use centralized path variables defined in the top cell
data_dump_folder = output_data_dump_folder
timestamp_file = session_timestamp_file

new_table = {"Stock": [], "Date": [], 'Close': [], "S_EMA1": [], "S_EMA2": [], "C_EMA1" : [], "C_EMA2": [], "S_RSI_P": [], "S_RSI_L_R": [], 'C_RSI_V': [],  'P_Pos': [], 'C_Pos': []}
df = pd.DataFrame(new_table)

# Initialize as None - will be calculated when first needed
today_date_str = None
yesterday_date_str = None

# --- Configuration ---
LOGIN_URL = login_url

# --- IMPORTANT: Replace with your actual username and password ---
USERNAME = "WK178100"
PASSWORD = os.getenv("My_ICICI_Password")

# --- CRITICAL: UPDATE THESE SELECTORS BASED ON YOUR WEBSITE'S HTML ---

# Selector for the initial "LOGIN" button that reveals the form (first page)
INITIAL_LOGIN_BUTTON_SELECTOR_TYPE = By.XPATH
INITIAL_LOGIN_BUTTON_SELECTOR_VALUE = "//a[contains(text(), 'LOGIN')]"

# Selector for the "View Apps" button after OTP/login completion
VIEW_APPS_BUTTON_SELECTOR_TYPE = By.ID
VIEW_APPS_BUTTON_SELECTOR_VALUE = "pills-view-tab"

# Selector for the "Login" link after "View Apps" (opens new tab/window)
NEXT_LINK_AFTER_VIEW_APPS_SELECTOR_TYPE = By.XPATH
NEXT_LINK_AFTER_VIEW_APPS_SELECTOR_VALUE = "//a[@href='https://api.icicidirect.com/apiuser/login?api_key=X4Cx7n2k69498TI15%3d753L0H3%2b4V2474' and contains(text(), 'Login') and @target='_blank']"

# --- Selectors for the SECOND Login Page (the new tab/window) ---
# Username field on the second login page
SECOND_LOGIN_USERNAME_FIELD_SELECTOR_TYPE = By.ID
SECOND_LOGIN_USERNAME_FIELD_SELECTOR_VALUE = "txtuid"

# Password/PIN field on the second login page
SECOND_LOGIN_PASSWORD_FIELD_SELECTOR_TYPE = By.ID
SECOND_LOGIN_PASSWORD_FIELD_SELECTOR_VALUE = "txtPass"

# Terms and Conditions checkbox on the second login page
SECOND_LOGIN_TNC_CHECKBOX_SELECTOR_TYPE = By.ID
SECOND_LOGIN_TNC_CHECKBOX_SELECTOR_VALUE = "chkssTnc"

# FINAL SUBMIT BUTTON on this SECOND Login Page (the one that triggers the second OTP)
SECOND_LOGIN_FINAL_SUBMIT_BUTTON_SELECTOR_TYPE = By.ID
SECOND_LOGIN_FINAL_SUBMIT_BUTTON_SELECTOR_VALUE = "btnSubmit"

# Submit button on the first OTP popup
FIRST_OTP_SUBMIT_BUTTON_SELECTOR_TYPE = By.ID
FIRST_OTP_SUBMIT_BUTTON_SELECTOR_VALUE = "btnsubotp"

# Submit button on the second OTP popup (after submitting the second login form)
SECOND_OTP_SUBMIT_BUTTON_SELECTOR_TYPE = By.ID
SECOND_OTP_SUBMIT_BUTTON_SELECTOR_VALUE = "Button1"

# Path to your WebDriver executable (e.g., chromedriver.exe or geckodriver.exe)
DRIVER_PATH = webdriver_path

# Choose your browser: 'chrome' or 'firefox'
BROWSER: str = 'chrome'

# Max time to wait for elements to be present (in seconds)
WAIT_TIME: int = 15  # General wait time for most elements

# Dedicated wait times for manual OTP entry (both set to 30 seconds)
FIRST_OTP_MANUAL_ENTRY_WAIT_TIME: int = 25
SECOND_OTP_MANUAL_ENTRY_WAIT_TIME: int = 25
# --- End Configuration ---

# ==== Function call from within get_today_date_str() and get_yesterday_date_str() =====
def _initialize_dates():
    """Initialize both today and yesterday date strings using IST timezone"""
    global today_date_str, yesterday_date_str
    if today_date_str is None or yesterday_date_str is None:
        # Get today's date from IST timezone
        today_date_str, _ = ist_date_time()
        # Calculate yesterday using IST timezone
        india_timezone = pytz.timezone('Asia/Kolkata')
        utc_now = datetime.now(timezone.utc)
        ist_now = utc_now.astimezone(india_timezone)
        yesterday_ist = ist_now - timedelta(days=1)
        yesterday_date_str = yesterday_ist.date().strftime('%Y-%m-%d')

# ==== Function call from within _initialize_dates() =====
def ist_date_time():
    # Get the Indian/Maldives timezone object (which covers IST)
    india_timezone = pytz.timezone('Asia/Kolkata')
    # Get the current time in UTC
    utc_now = datetime.now(timezone.utc) # **Calling datetime.now, returns datetime object**
    # Localize the UTC time to the Indian timezone
    ist_now = utc_now.astimezone(india_timezone) # **Calling astimezone, returns datetime object**
    # Return the date part as a string and the time as a string
    return ist_now.date().strftime('%Y-%m-%d'), ist_now.time().strftime("%H:%M:%S") # **Calling date() and time() methods, returns date and time objects, then strftime() returns string**


def get_today_date_str():
    """Lazy initialization of today_date_str to avoid delay during imports"""
    global today_date_str
    if today_date_str is None:
        _initialize_dates()
    return today_date_str


def get_yesterday_date_str():
    """Lazy initialization of yesterday_date_str using IST timezone"""
    global yesterday_date_str
    if yesterday_date_str is None:
        _initialize_dates()
    return yesterday_date_str


def setup_isec_login():
    global s_token
    
    filepath = timestamp_file
    todays_date = get_today_date_str()  # Use lazy initialization

    if os.path.exists(filepath):

      token, date = extract_token_date() # **Calling extract_token_date, returns str, str**

      if date == todays_date:
        s_token = token
        print("Token already exists.\n")
      else:
        print("Hi. Since the current token is obsolete we need to generate new token. Going there....\n")
        tm.sleep(3) # **Calling tm.sleep, takes int or float**

        s_token = automate_login()

    print(f"The generated token number is {s_token}. \n")

    isec, login_outcome = isec_login(s_token)
    return isec, login_outcome


# ==== Function call from within setup_isec_login() =====
def extract_token_date():
    # Specify the filename
    file_name = timestamp_file # Make sure this matches the filename used to save
    try:
        # Open the file in read mode ('r')
        with open(file_name, "r") as f:
            # Read the entire content of the file
            file_content = f.read()
        # The content is expected to be in the format "session_key,date"
        # Split the string by the comma
        parts = file_content.split(',')
        # The date is the second part (index 1)
        if len(parts) > 1:
            extracted_token_str = parts[0]
            extracted_date_str = parts[1]
            return extracted_token_str, extracted_date_str
        else:
            print("Could not find the file content to separate the session key and date.")
            sys.exit()
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")
        sys.exit()
    except Exception as e:
        print(f"An error occurred during the token date extraction : {e}")
        sys.exit()


# ==== Function call from within setup_isec_login() =====
def automate_login() -> None | str:
    driver = None
    original_window = None
    api_session_key = None
    try:
        if BROWSER == 'chrome':
            service = ChromeService(executable_path=DRIVER_PATH)
            driver = webdriver.Chrome(service=service)
        elif BROWSER == 'firefox':
            service = FirefoxService(executable_path=DRIVER_PATH)
            driver = webdriver.Firefox(service=service)
        else:
            #print("Unsupported browser specified. Please choose 'chrome' or 'firefox'.")
            return None
        original_window = driver.current_window_handle
        #print(f"Opening {LOGIN_URL} in {BROWSER} browser...")
        driver.get(LOGIN_URL)
        wait = WebDriverWait(driver, WAIT_TIME)
        # --- Step 1: Click the initial LOGIN button (first page) ---
        #print("Looking for the initial LOGIN button...")
        try:
            initial_login_button = wait.until(EC.element_to_be_clickable((INITIAL_LOGIN_BUTTON_SELECTOR_TYPE, INITIAL_LOGIN_BUTTON_SELECTOR_VALUE)))
            initial_login_button.click()
            #print("Initial LOGIN button clicked. Waiting for login form to appear...")
            tm.sleep(3)
        except Exception as e:
            #print(f"Error: Initial LOGIN button not found or not clickable: {e}")
            #print("Please confirm the selector for the initial button that reveals the login form.")
            return None
        # --- Step 2: Fill in Username and Password on the FIRST login form ---
        #print("Attempting to interact with the first login form (if present)...")
        try:
            # Added more specific waits for presence and visibility
            username_field = wait.until(EC.visibility_of_element_located((By.ID, "txtuid")))
            username_field.send_keys(USERNAME)
            #print("Username entered on first form.")
            password_field = wait.until(EC.visibility_of_element_located((By.ID, "txtPass")))
            password_field.send_keys(PASSWORD)
            #print("Password entered on first form.")
            final_login_button = wait.until(EC.element_to_be_clickable((By.ID, "btnlogin")))
            final_login_button.click()
            #print("Final Login button on first form clicked.")
            tm.sleep(5) # Give time for page to process
        except (NoSuchElementException, TimeoutException): # Catching specific exceptions for this optional step
            #print("First login form fields/button not found. Proceeding as if this step is not always present or handled by a modal.")
            pass # Removed 'except Exception as e:' from inside, as it was incorrectly indented
        except Exception as e: # This is the correct placement for a broader catch
            #print(f"Error during first login form interaction: {e}")
            return None
        # --- Pause for manual OTP entry (first time) & Click Submit ---
        #print("\n!!! FIRST OTP POPUP DETECTED !!!")
        #print("Please manually enter the 6-digit OTP sent to your phone in the browser window.")
        #print(f"Pausing script for {FIRST_OTP_MANUAL_ENTRY_WAIT_TIME} seconds to allow for OTP entry.")
        tm.sleep(FIRST_OTP_MANUAL_ENTRY_WAIT_TIME) # Manual OTP entry pause
        #print("Attempting to click 'Submit' button on first OTP popup...")
        try:
            first_otp_submit_button = wait.until(
                EC.element_to_be_clickable((FIRST_OTP_SUBMIT_BUTTON_SELECTOR_TYPE, FIRST_OTP_SUBMIT_BUTTON_SELECTOR_VALUE))
            )
            first_otp_submit_button.click()
            #print("First OTP 'Submit' button clicked.")
            tm.sleep(5)
        except (NoSuchElementException, ElementClickInterceptedException, TimeoutException) as otp1_e:
            #print(f"Error: First OTP Submit button (ID: '{FIRST_OTP_SUBMIT_BUTTON_SELECTOR_VALUE}') not found, not clickable, or timed out.")
            #print(f"Details of first OTP button error: {otp1_e}")
            #print("Please confirm the selector for the FIRST OTP submit button (`btnsubotp`) and that it becomes visible and clickable after OTP entry.")
            return None
        # --- Step 5: Click the "View Apps" button ---
        #print("Looking for 'View Apps' button...")
        try:
            view_apps_button = wait.until(EC.element_to_be_clickable((VIEW_APPS_BUTTON_SELECTOR_TYPE, VIEW_APPS_BUTTON_SELECTOR_VALUE)))
            view_apps_button.click()
            #print("'View Apps' button clicked. Giving more time for subsequent elements to load...")
            tm.sleep(10) # Increased sleep to accommodate potential overlays after View Apps click
        except Exception as e:
            #print(f"Error: 'View Apps' button not found or not clickable: {e}")
            #print("Please confirm the selector for the 'View Apps' button. It might not be visible or clickable immediately after OTP.")
            return None
        # --- Step 6: Click the "Login" link after "View Apps" (opens new tab) ---
        #print("Looking for the 'Login' link after 'View Apps' (which opens a new tab)...")
        try:
            old_window_handles = driver.window_handles
            next_link_button = wait.until(EC.element_to_be_clickable((NEXT_LINK_AFTER_VIEW_APPS_SELECTOR_TYPE, NEXT_LINK_AFTER_VIEW_APPS_SELECTOR_VALUE)))
            # --- Handle ElementClickInterceptedException with JavaScript fallback ---
            try:
                next_link_button.click() # Attempt normal click first
                #print("'Login' link after 'View Apps' clicked.")
            except ElementClickInterceptedException:
                #print("Element click intercepted, trying JavaScript click for 'Login' link...")
                driver.execute_script("arguments[0].click();", next_link_button)
                #print("JavaScript click executed for 'Login' link.")
            # --- End JavaScript fallback ---
            tm.sleep(5) # Give browser time to open new tab/window
            # --- Switch to the new window/tab (using simpler logic) ---
            wait.until(EC.number_of_windows_to_be(2))
            driver.switch_to.window(driver.window_handles[-1])
            #print("Switched to the new login window.")
            tm.sleep(7) # Give new page ample time to load all elements
        except Exception as e:
            #print(f"Error: 'Login' link after 'View Apps' not found or not clickable, or failed to switch window: {e}")
            #print("Please re-inspect the 'Login' link *after* clicking 'View Apps'. The selector might have changed or the element might not be visible/clickable immediately.")
            return None
        # --- Step 7: Fill in Username and Password on the SECOND login page ---
        #print("Looking for username field on SECOND login page...")
        try:
            second_login_username_field = wait.until(EC.presence_of_element_located((SECOND_LOGIN_USERNAME_FIELD_SELECTOR_TYPE, SECOND_LOGIN_USERNAME_FIELD_SELECTOR_VALUE)))
            second_login_username_field.send_keys(USERNAME)
            #print("Username entered on second login page.")
        except Exception as e:
            #print(f"Error: Username field on second login page (ID: '{SECOND_LOGIN_USERNAME_FIELD_SELECTOR_VALUE}') not found or not interactable: {e}")
            #print("Please confirm the selector for the username input field on the second login page.")
            return None
        #print("Looking for password field on SECOND login page...")
        try:
            second_login_password_field = wait.until(EC.presence_of_element_located((SECOND_LOGIN_PASSWORD_FIELD_SELECTOR_TYPE, SECOND_LOGIN_PASSWORD_FIELD_SELECTOR_VALUE)))
            second_login_password_field.send_keys(PASSWORD)
            #print("Password entered on second login page.")
        except Exception as e:
            #print(f"Error: Password field on second login page (ID: '{SECOND_LOGIN_PASSWORD_FIELD_SELECTOR_VALUE}') not found or not interactable: {e}")
            #print("Please confirm the selector for the password input field on the second login page.")
            return None
        # --- Step 8: Check the "I agree to the terms and conditions" box ---
        #print("Looking for T&C checkbox on SECOND login page...")
        try:
            tnc_checkbox = wait.until(EC.element_to_be_clickable((SECOND_LOGIN_TNC_CHECKBOX_SELECTOR_TYPE, SECOND_LOGIN_TNC_CHECKBOX_SELECTOR_VALUE)))
            if not tnc_checkbox.is_selected():
                tnc_checkbox.click()
                #print("T&C checkbox clicked.")
            else:
                #print("T&C checkbox already checked.")
                pass
        except Exception as e:
            #print(f"Error: T&C checkbox on second login page (ID: '{SECOND_LOGIN_TNC_CHECKBOX_SELECTOR_VALUE}') not found or not clickable: {e}")
            #print("Please confirm the selector for the 'I agree to the terms and conditions' checkbox.")
            return None
        # --- Step 9: Click the final submit button on the SECOND login page ---
        #print("Looking for the FINAL SUBMIT button on SECOND login page...")
        try:
            second_login_submit_button = wait.until(EC.element_to_be_clickable((SECOND_LOGIN_FINAL_SUBMIT_BUTTON_SELECTOR_TYPE, SECOND_LOGIN_FINAL_SUBMIT_BUTTON_SELECTOR_VALUE)))
            second_login_submit_button.click()
            #print("FINAL SUBMIT button on second login page clicked.")
            tm.sleep(5) # Give more time for the next OTP popup to appear and load fully
            # --- Pause for manual OTP entry (second time) & Click Submit ---
            #print("\n!!! SECOND OTP POPUP DETECTED !!!")
            #print("Please manually enter the 6-digit OTP sent to your phone for the second login.")
            #print(f"Pausing script for {SECOND_OTP_MANUAL_ENTRY_WAIT_TIME} seconds to allow for OTP entry.")
            tm.sleep(SECOND_OTP_MANUAL_ENTRY_WAIT_TIME) # Manual OTP entry pause
            #print("Looking for 'Submit' button on second OTP popup...")
            try:
                # Increased specific wait time for this critical button
                otp_submit_button = WebDriverWait(driver, WAIT_TIME * 2).until(
                    EC.element_to_be_clickable((SECOND_OTP_SUBMIT_BUTTON_SELECTOR_TYPE, SECOND_OTP_SUBMIT_BUTTON_SELECTOR_VALUE))
                )
                otp_submit_button.click()
                #print("Second OTP 'Submit' button clicked.")
                tm.sleep(7) # Give sufficient time for redirection after OTP submission
            except (NoSuchElementException, ElementClickInterceptedException, TimeoutException) as otp_e:
                #print(f"Error: OTP Submit button (ID: '{SECOND_OTP_SUBMIT_BUTTON_SELECTOR_VALUE}') not found, not clickable, or timed out.")
                #print(f"Details of OTP button error: {otp_e}")
                #print("Please confirm the selector for the OTP submit button (`Button1`) and that it becomes visible and clickable after OTP entry.")
                return None
            # --- Fetch API Session Key ---
            #print("Attempting to fetch API session key from current URL using polling...")
            max_attempts = 20 # Check for 20 seconds
            api_session_key_found = False
            for i in range(max_attempts):
                current_url = driver.current_url
                #print(f"Attempt {i+1}/{max_attempts}: Current URL: {current_url}")
                parsed_url = urllib.parse.urlparse(current_url)
                query_params = urllib.parse.parse_qs(parsed_url.query)
                if 'apisession' in query_params and query_params['apisession']:
                    api_session_key = query_params['apisession'][0]
                    #print(f"\n--- API Session Key Found: {api_session_key} ---\n")
                    api_session_key_found = True
                    break
                tm.sleep(1)
            if not api_session_key_found:
                #print("API Session Key not found in the URL after polling. Final URL observed:")
                #print(current_url)
                api_session_key = None # Ensure it's None if not found after all attempts
        except Exception as e:
            #print(f"An unexpected error occurred during final submit, OTP interaction, or API key fetch: {e}")
            #print(f"Details: {e}")
            return None
        #print("Login and navigation up to second OTP submission completed. Browser will close shortly.")
        tm.sleep(7)
    except WebDriverException as e:
        #print(f"WebDriver error: {e}")
        #print("Please ensure your WebDriver (chromedriver.exe) is correctly installed at the specified path and its version matches your Chrome browser version.")
        #print("If the versions mismatch, you will see errors like 'Session not created: this version of ChromeDriver only supports Chrome version X'.")
        return None
    except Exception as e:
        #print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if driver:
            driver.quit()
            #print("Browser closed.")

    save_sessionkey_date(api_session_key)
    return api_session_key


# ==== Function call from within automate_login() =====
def save_sessionkey_date(api_session_key: None | str):

    # Get the current timestamp
    current_date = get_today_date_str()

    # Specify the filename
    file_name = timestamp_file # You can change the path and filename

    # Open the file in write mode ('w') - this will create the file if it doesn't exist or overwrite it if it does
    with open(file_name, "w") as f:
      # Combine the session key and timestamp into a single string and write to the file
      f.write(f"{api_session_key},{current_date}")
      print(f"Session key and date saved to {file_name}. \n")
      f.close()


# ==== Function call from within setup_isec_login() =====
def isec_login(s_token):

    try:
        isec = BreezeConnect(api_key="X4Cx7n2k69498TI15=753L0H3+4V2474")
        isec.generate_session(api_secret="319$5126u_2r37091nz4o51G79a49L+1", session_token = s_token)
        return isec, "Login Success"

    except Exception as e:
        return "Could not authenticate credentials. Please check session key.", "Try Again."



def fetch_hist_data(name_of_stock, isec, start_date_str, end_date_str):
    start_date = start_date_str
    end_date = end_date_str

    time_interval = "1day"
    exchange_name = "NSE"
    product_cat = "cash"
    data = isec.get_historical_data(interval = time_interval, from_date = start_date, to_date = end_date, stock_code = name_of_stock, exchange_code = exchange_name, product_type = product_cat)
    stock_data = pd.DataFrame(data["Success"])

    if stock_data.empty:
        return stock_data
    else:
        columns_to_drop = ['stock_code', 'exchange_code', 'product_type', 'expiry_date', 'right', 'strike_price', 'open_interest', 'count']
        stock_data.drop(columns=columns_to_drop, inplace=True)
        return stock_data

# ==== indirect Function call to convert today_date_str to yesterday_data_obj =====
def get_yesterdays_date():
    date_yesterday = (datetime.strptime(get_today_date_str(), '%Y-%m-%d').date() - timedelta(days=1)).strftime('%Y-%m-%d') # Then converting str to datetime object, subtracting timedelta, and formatting back to str.**
    return date_yesterday # **Returns str**


def hist_data_load_transform(stock_df):

    data = stock_df.rename(columns={"datetime": "Date", "close": "Close", "open": "Open", "high": "High", "low": "Low",
                                "volume": "Volume"})
    data['Date'] = pd.to_datetime(data['Date']).dt.date # **Converting to datetime and then extracting date object**
    data_new = data.sort_values(by = "Date") # **Sorting by date object**
    return data_new


################ Main execution block ################

# Initialize dates when first needed instead of immediately
get_today_date_str()  # This will set both today_date_str and yesterday_date_str

try:
    
    isec, login_outcome = setup_isec_login() #done # **Calling setup_isec_login**
    print(f"Login outcome: {login_outcome}\n")

    if login_outcome == "Login Success":

        print("Working on fetching historical data...\n")
        # Define the UTC timezone
        start_date_str: str = "2013-01-01"
        end_date_str: str = "2025-09-30"

        stock_df = fetch_hist_data(TICKER, isec, start_date_str, end_date_str)
        #print(stock_df)
        stock_df_new = hist_data_load_transform(stock_df) # **Calling hist_data_load_transform, returns DataFrame**

        stock_df_new.to_csv(os.path.join(data_dump_folder, f"{TICKER}_hist_daily_data.csv"), index=False)
        print("All historical daily data fetch done.")

except Exception as e:
    print("The error is : ", e)    
   

Token already exists.

The generated token number is 53308723. 

Login outcome: Login Success

Working on fetching historical data...

All historical daily data fetch done.
